In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
lending_fil = <path of file>
pd_lend_df = pd.read_csv(lending_fil)
pd_lend_df.info()

In [ ]:
pd_lend_df.dropna(axis = 1, how = 'all' ,inplace = True)
pd_lend_df.info()
#pd_numeric = pd_lend_df.describe()
#pd_numeric.info()

In [ ]:
pd_unique=pd_lend_df.loc[:,pd_lend_df.nunique()==1].copy()
pd_unique.describe(include = 'all')


In [ ]:
pd_lend_df=pd_lend_df.loc[:,pd_lend_df.nunique()>1]

In [ ]:
pd_lend_df.info()

In [ ]:
pd_lend_df.describe().to_csv(<path of to-Csv>)
#print(len(pd_lend_df['member_id'].unique()))
#print(len(pd_lend_df['id'].unique()))

In [ ]:
pd_lend_df.describe(include ='object')

In [ ]:
print(len(pd_lend_df[pd_lend_df['term'].str.contains("months") == True]))

In [ ]:
pd_lend_df['term'] = pd_lend_df['term'].str.replace("months" ,"")
pd_lend_df.describe(include ='object')

In [ ]:
print(len(pd_lend_df[pd_lend_df['int_rate'].str.contains("%") == True]))

In [ ]:
pd_lend_df['int_rate'] = pd_lend_df['int_rate'].str.replace("%" ,"")
pd_lend_df.describe(include ='object')

In [ ]:
print(len(pd_lend_df[pd_lend_df['revol_util'].str.contains("%") == True]))

In [ ]:
pd_lend_df['revol_util'] = pd_lend_df['revol_util'].str.replace("%" ,"")
pd_lend_df.describe(include ='object')

In [ ]:
print(len(pd_lend_df[pd_lend_df['emp_length'].str.contains("years") == True]))
print(pd_lend_df['emp_length'].unique())

In [ ]:
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("< 1" ,"0")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("+" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("years" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("year" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace(" " ,"")
print(pd_lend_df['emp_length'].unique())
#pd_lend_df.describe(include ='object')



In [ ]:
pd_lend_df['zip_code'] = pd_lend_df['zip_code'].str.replace("xx" ,"")
pd_lend_df.describe(include ='object')

In [ ]:
print(pd_lend_df['next_pymnt_d'].unique())
print(pd_lend_df['addr_state'].unique())
print(pd_lend_df['purpose'].unique())  # might be imp
print(pd_lend_df['verification_status'].unique())
print(pd_lend_df['home_ownership'].unique())
print(pd_lend_df['grade'].unique())
print(pd_lend_df['sub_grade'].unique())
print(pd_lend_df['loan_status'].unique())

In [ ]:
print(len(pd_lend_df['url'].unique()))
len_url = pd_lend_df['url'].str.len()
len_url.describe()

In [ ]:
len_desc = pd_lend_df['desc'].str.len()
len_desc.describe()

In [ ]:
#from urllib.parse import urlparse
import urllib.parse as urlparse
pd_lend_df['protocol'],pd_lend_df['domain'],pd_lend_df['path'],pd_lend_df['query'],pd_lend_df['fragment'] = zip(*pd_lend_df['url'].map(urlparse.urlsplit))
#pd_lend_df['url']= zip(*pd_lend_df['url'].map(urlparse.urlsplit))[3]
#print(pd_lend_df['protocol'].unique())
#print(pd_lend_df['domain'].unique())
#print(pd_lend_df['path'].unique())
pd_lend_df['query'] = pd_lend_df['query'].str.replace("loan_id=","")
#print(pd_lend_df['query'].unique())
#print(pd_lend_df['fragment'].unique())
pd_lend_df.drop(columns = ['protocol','domain','path','fragment','url'] ,inplace = True)
pd_lend_df.describe(include ='object')

In [ ]:
#rint(len(pd_lend_df['query'].unique()))
#print(pd_lend_df['query'].unique())

In [ ]:
pd_lend_df['desc'].head()


In [ ]:
pd_lend_df['desc'].tail()

In [ ]:
pd_lend_df['last_pymnt_d'].unique()

In [ ]:
pd_lend_df.corr().to_csv(<corr file path>)


In [ ]:
pd_lend_df.drop(columns=['desc','funded_amnt_inv','out_prncp_inv','total_pymnt_inv','total_pymnt_inv','total_rec_prncp','total_rec_int',
                        'next_pymnt_d','query','member_id'], inplace = True)
#desc is long text column, some of this info is in purpose and tile tolumns
#funded_amnt_inv , loan amt and funded amt are highly correlated, dropping one for now
#out_prncp_inv is highly correlated out_prncp; similar case for total_pymnt_inv ,total_pymnt_inv
# total_pymnt = total_rec_prncp + total_rec_int , hence dropping princ and int cols
# next_pymnt_d has lot of blanks and same vals , drop
#query and # id same , drop query , member_id all unique
pd_lend_df.to_csv(<post cleanup file>,index = False)


In [ ]:
fig = px.box(pd_lend_df, y="loan_amnt")
fig.update_traces(quartilemethod="linear") # or "inclusive", or "linear" by default
fig.show()

In [ ]:
fig = px.histogram(pd_lend_df, x="loan_amnt")
fig.show()

In [ ]:
fig = px.histogram(pd_lend_df, x="loan_status")
fig.show()